In [ ]:
# Imports
from ultralytics import YOLO
import os
from pathlib import Path
import skimage as ski
import AMMVRP 

#### Step 0: Create YOLO 1.1 formatted directory and files

In [ ]:
class_names = ['your','class','names']

# Create directory
AMMVRP.create_yolo_1_1_dir(class_names)

# Example:
# create_yolo_1_1_dir(
#     class_names = ["TNK", "APC", "ICBM", "IMV", "SPH", "MLRS",
#     "SRBM", "UAVC", "SAM", "IFV", "CT", "SC"
#     ]
# )

In [ ]:
# After adding images to your dataset directories, below lines create <<'val' | 'train' | 'test'>>.txt files.
AMMVRP.gen_txt_file('train')
#gen_txt_file('val')
#gen_txt_file('test')

### Step 1: Train the YOLO v11 Model with Validation
> - Load base YOLO model (v11) and begin training model on custom dataset

In [ ]:
#train_num = '' # if the latest run is not blank

model = YOLO(
    "yolo11n.pt"  # to start custom dataset training, the lightest version of v11, n = nano
    #f'{AMMVRP.proj_path}/runs/detect/train{train_num}/weights/last.pt'  # if resuming last iteration of training
    )  
results = model.train(
    data="config.yaml", 
    epochs=25, 
    device='mps', # mps: Apple silicon GPU
    imgsz = 640,
    val = True,
    patience = 75, # will stop training at X epochs if performance isn't improving significantly
    #### Augmentation Parameters ####
    hsv_h=1.0,  # Hue Adjustment
    hsv_s=1.0,  # Saturation Adjustment
    hsv_v=1.0,  # Brightness Adjustment
    degrees = 180, # Rotations
    pretrained=True #load previous weights, not resuming
    #translate=0.0,
    #scale=0.0,
    #fliplr=0.0,
    #mosaic=0.0,
    #erasing=0.0,
    #auto_augment=None,
    #### Other Parameters ####
    #resume = True  # if resuming last iteration of training
    ) 

#### Step 1.5: Auto Annotate Images for val and test datasets

In [ ]:
dataset = 'val' #'val' | 'test'
train_num = '' # number of "train" directory with most updated weights

auto_annotate_no_SAM(
    data=f'{AMMVRP.proj_path}/auto_annotate/{dataset}',  # folder or list of image paths
    det_model=f'{AMMVRP.proj_path}/runs/detect/train{train_num}/weights/best.pt',  # pretrained detection model
    device="mps",  # or "cpu"
    output_dir=f'{AMMVRP.proj_path}/auto_annotate/test/labels',
    conf=.6,
    max_det = 5
)

#### Step 2: Validation

In [ ]:
# Run validation and set parameters
#conf=.35
iou=0
model = YOLO(f'{AMMVRP.proj_path}/runs/detect/train/weights/best.pt')  # load a custom model

for conf in [.2,.35,.5,.7]:
    metrics = model.val(
        data="config.yaml", # to use test dataset for post-training model comparison ('test.yaml')
        imgsz = 640,
        conf=conf,
        iou=iou, # Intersection Over Union (IoU) threshold for Non-Maximum Suppression (NMS). range= 0(no overlap) - 1 (overlap)
        max_det=5, # max_num detections
        device='mps', # mps=Apple silicon, default='cpu' 
        augment=True,  # Enables test-time augmentation (TTA) for predictions, improves detection robustness at the cost of inference speed.
        plots = True,
        project = 'val_runs',
        name = f'conf_{conf}_iou{iou}',
        verbose = True
    )

#### Step 3: Test

In [ ]:
# Test model with "test" dataset and find resulting, annotated images in ../runs/detect/predict

train_num = '' # number of "train" directory with most updated weights

model = YOLO(f'{AMMVRP.proj_path}/runs/detect/train{train_num}/weights/best.pt')  # load most updated YOLO model

# Run test and set parameters
conf=.35
iou=0

results = model.predict(
    source=f'{AMMVRP.proj_path}/test', 
    conf=conf, # minimum confidence threshold
    iou=iou, # Intersection Over Union (IoU) threshold for Non-Maximum Suppression (NMS). range= 0(no overlap) - 1 (overlap)
    max_det=5, # max_num detections
    device='mps', # mps=Apple silicon, default='cpu'
    name=f'predict_conf_{conf}_iou_{iou}',
    save=True, 
    augment=True  # Enables test-time augmentation (TTA) for predictions, improves detection robustness at the cost of inference speed.
)